# Homework 3. Programming in miniKanren

Evgeny Bobkunov

SD-03

e.bobkunov@innopolis.university



In type systems, typing judgements are formal statements about expressions of the form

$\Gamma  \vdash e : T$

This notation reads “in context $\Gamma$, expression $e$ has type $T$”. A context is simply a list (dictionary)
of typings for local variables. For example, consider the following instance of a typing judgement:

$x : \mathsf{Int}, f : \mathsf{Int} \rightarrow \mathsf{Bool} \vdash f (x+x) : \mathsf{Bool}$

Here, $x$ has type $\mathsf{Int}$ and $f$ is a function from $\mathsf{Int}$ to $\mathsf{Bool}$. The context tells us the types of local variables $x$ and $f$ that are used in expression $f (x+x)$. The overall judgement says that this expression
in this context has type $\mathsf{Bool}$.

In the following exercises, you will gradually implement the ternary relation corresponding to such
typing judgements:

`(typed-expro context expr type)`

The expressive power of logical programming comes from the ability to use a given relation in
different “modes”, depending on which arguments of the relation are known at the type of execution.
Given proper conditions, we can hope that implementation of this relation using logical programming
in miniKanren will immediately (or with minor additional effort) provide some useful modes:

1. If context, expression, and type are known — we are performing **type checking**. This is usually
very straightforward, and so we would like to mainly have this in mind during implementation.

2. If context and expression are known, but type is not — executing this relation in miniKanren
*should* correspond to **type inference**. Since many type inference algorithms rely on unification,
and unification is built into logical programming, we hope to get this essentially for free.

3. If context and type are known, but expression is not — executing this relation in miniKanren
*should* correspond to **program synthesis**. While practical approaches to program synthesis
normally require a more careful pruning of the search space (even for type-directed synthesis),
we hope to at least get some simple programs to synthesize.

Although there are multiple caveats, this, perhaps, na¨ıve idea of implementing checking and then
getting both type inference and program synthesis almost for free seems very appealing. In the following
series of exercises, you will begin the exploration with just some simple expressions and types.


## 3.1 Contexts

To start things off, let’s implement some useful helpers to work with contexts. Contexts are merely
lists of pairs, where each pair consists of a variable identifier (e.g. `x`) and its type (e.g. `Int`). For
convenience, in addition to variables we will also allow operators (e.g. `+`) and numeric constants (e.g.
`123`) to appear in contexts.

In the following exercises, we will use only simple types:

1. `'Bool` and `'Int` will be used to typecheck some built-in expressions.
2. If `t1` and `t2` are types, then `(t1 -> t2)` is a function type for a function that is supposed to
take a value of type `t1` as input and produce a value of type `t2`. Function types may be nested,
but unlike Haskell, we will not provide associativity:
- well-formed function types:
    - `'(Int -> Bool)`
    - `'(Int -> (Int -> Int))`
    - `'((a -> b) -> ([a] -> [b]))`
- ill-formed function types (missing parentheses):
    - `'(Int -> Int -> Int)`
    - `'((a -> b) -> [a] -> [b])`

3. Anything else (e.g. `'a`, `'b`, `'[a]`) will be seen as an uninterpreted type. That is, we allow this
type, but it has no particular meaning (to us).

**Exercise 3.1.1**

Implement a miniKanren relation `context-lookupo` such that
`(context-lookupo var type context)` is satisfied when `context` has an explicit knowledge that `var`
has type type. Check your implementation:

```
(define sample-context
  '((x . Int)
    (f . (Int -> Bool))
    (y . Int)
    (b . Bool)
    (+ . (Int -> (Int -> Int)))))

(run* (type) (context-lookupo sample-context 'x type))
; '(Int)
(run* (type) (context-lookupo sample-context 'z type))
; '()
(run* (var) (context-lookupo sample-context var 'Int))
; '(x y)
(run* (var type) (context-lookupo sample-context var type))
; '((x Int) (f (Int -> Bool)) (y Int) (b Bool) (+ (Int -> (Int -> Int))))
```

In [13]:
(require minikanren)
(require minikanren/matche)

(define sample-context
  '((x . Int)
    (f . (Int -> Bool))
    (y . Int)
    (b . Bool)
    (+ . (Int -> (Int -> Int)))))

(define (context-lookupo context var type)
  (fresh (rest-context pair)
    (== context (cons pair rest-context))
    (conde
      [(== (cons var type) pair)]
      [(context-lookupo rest-context var type)])))

In [15]:
(run* (type) (context-lookupo sample-context 'x type))

'(Int)

In [17]:
(run* (type) (context-lookupo sample-context 'z type))

'()

In [19]:
(run* (var) (context-lookupo sample-context var 'Int))

'(x y)

In [21]:
(run* (var type) (context-lookupo sample-context var type))

'((x Int) (f (Int -> Bool)) (y Int) (b Bool) (+ (Int -> (Int -> Int))))

**Exercise 3.1.2**

Implement a miniKanren relation `context-varso`
such that
`(context-varso context vars)` is satisfied when `context` has contains exactly `vars`. Check your
implementation:

```
(run* (vars) (context-varso sample-context vars))
; '((x f y b +))
(run* (context) (context-varso context '(x y z)))
; '(((x . _.0) (y . _.1) (z . _.2)))
```

In [24]:
(define (context-varso context vars)
  (conde
    [(== context '())
     (== vars '())]
    [(fresh (var type rest-context rest-vars)
       (== context (cons `(,var . ,type) rest-context))
       (== vars (cons var rest-vars))
       (context-varso rest-context rest-vars))]))

In [26]:
(run* (vars) (context-varso sample-context vars))

'((x f y b +))

In [28]:
(run* (context) (context-varso context '(x y z)))

'(((x . _.0) (y . _.1) (z . _.2)))

## 3.2 Untyped Expressions

Before moving on to typing judgements, let us first implement a simplified version in the untyped
setting. Removing types means that we only need to know which variables (and constants) are available.
For example:

$x, y, f \vdash f (x + x)$

Here, expression $f (x + x)$ is well-formed in the context with variables `x`, `y` and `f`. This is because the
expression only uses variables `x` and `f`, both of which are present in the context. The variable `y` is not
used, but for now this is not a problem.

**Exercise 3.2.1**

Implement a miniKanren relation `untyped-expro`
such that
`(untyped-expro vars expr)` is satisfied when `expr` is a valid expression with variables from `vars`:

1. If a variable (or constant) is used in `expr`, it must be present in `vars`. However, if a variable (or
constant) is present in `vars` is does not have to be used in `expr`.
2. An expression is one of the following:
   - a variable: $x, y, z, . . .$
   - a constant (natural) number: 0, 1, 2, . . .
   - a binary operation: $(e_1 + e_2)$ or $(e_1 \times e_2)$
   - a unary function application: $(f x)$
   - a conditional expression: if $e_1$ then $e_2$ else $e_3$

Check your implementation:

Make sure that `untyped-expro` is capable of generating expressions that uses variables (and
constants) from a given list:

```
(run 6 (expr) (untyped-expro '(x) expr))
; '(x (x + x) (x x) (x * x) (if x then x else x) (x + (x + x)))
(run 8 (expr) (untyped-expro '(x y) expr))
; '(x y (x + x) (x * x) (x + y) (x x) (y x) (if x then x else x))
```

Make sure that `untyped-expro`
is capable of extracting a list of variables that is used in a given
expression:

```
(run 1 (vars) (untyped-expro vars
'(if (p x) then (x * (f (x + 1))) else (f (f x)))))
; '((p x f 1 . _.0))
```

*Hint: use `symbolo` and `numbero` to check if expression is a variable identifier or a numeric constant.*
*Hint: you might want to use `membero` and/or `not-membero`.*

In [32]:
(define (membero x l)
  (fresh (rest)
    (conde
      [(== l (cons x rest))]
      [(fresh (y)
         (== l (cons y rest))
         (membero x rest))])))

(define (untyped-expro vars expr)
  (conde
    [(symbolo expr)
     (membero expr vars)]
    
    [(numbero expr)]
    
    [(fresh (e1 e2 op)
       (== expr `(,e1 ,op ,e2))
       (membero op '(+ *))
       (untyped-expro vars e1)
       (untyped-expro vars e2))]
    
    [(fresh (f arg)
       (== expr `(,f ,arg))
       (symbolo f)
       (membero f vars)
       (untyped-expro vars arg))]
    
    [(fresh (test-expr then-expr else-expr)
       (== expr `(if ,test-expr then ,then-expr else ,else-expr))
       (untyped-expro vars test-expr)
       (untyped-expro vars then-expr)
       (untyped-expro vars else-expr))]))

In [34]:
(run 6 (expr) (untyped-expro '(x) expr))

'((_.0 (num _.0)) x ((_.0 + _.1) (num _.0 _.1)) ((x _.0) (num _.0)) ((_.0 + x) (num _.0)) ((if _.0 then _.1 else _.2) (num _.0 _.1 _.2)))

In [36]:
(run 8 (expr) (untyped-expro '(x y) expr))

'((_.0 (num _.0)) x y ((_.0 + _.1) (num _.0 _.1)) ((x _.0) (num _.0)) ((_.0 + x) (num _.0)) ((if _.0 then _.1 else _.2) (num _.0 _.1 _.2)) ((x + _.0) (num _.0)))

In [38]:
(run 1 (vars) (untyped-expro vars
'(if (p x) then (x * (f (x + 1))) else (f (f x)))))

'((p x f . _.0))

To bound generation of expressions, we can use relational arithmetic from `minikanren/numbers`:

**Exercise 3.2.2**

Implement a miniKanren relation `bounded-untyped-expro`
such that
`(bounded-untyped-expro max-depth vars expr)` is satisfied when `expr` is a valid expression with
variables from `vars` and depth less than or equal to `max-depth`. Check your implementation:

```
(run* (expr) (bounded-untyped-expro (build-num 2) '(x) expr))
; '(x (x x) (x + x) (x * x) (if x then x else x))
(run 1 (vars) (bounded-untyped-expro (build-num 5) vars
'(if (p x) then (x * (f (x + 1))) else (f (f x)))))
; '((p x f 1 . _.0))
```

In [41]:
(define (build-num n)
  (if (zero? n)
      '()
      (cons 1 (build-num (sub1 n)))))

(define (less-than-or-equalo n m)
  (conde
    [(== n '())]
    [(fresh (n-rest m-rest)
       (== n (cons 1 n-rest))
       (== m (cons 1 m-rest))
       (less-than-or-equalo n-rest m-rest))]))

(define (membero x l)
  (fresh (rest)
    (conde
      [(== l (cons x rest))]
      [(fresh (y)
         (== l (cons y rest))
         (membero x rest))])))

(define (bounded-untyped-expro max-depth vars expr)
  (conde
    [(symbolo expr)
     (membero expr vars)
     (less-than-or-equalo (build-num 1) max-depth)]
    
    [(numbero expr)
     (less-than-or-equalo (build-num 1) max-depth)]
    
    [(fresh (e1 e2 op depth1)
       (== expr `(,e1 ,op ,e2))
       (membero op '(+ *))
       (fresh (depth1-rest)
         (== depth1 (cons 1 depth1-rest))
         (less-than-or-equalo depth1 max-depth)
         (bounded-untyped-expro depth1-rest vars e1)
         (bounded-untyped-expro depth1-rest vars e2)))]
    
    [(fresh (f arg depth1 depth1-rest)
       (== expr `(,f ,arg))
       (symbolo f)
       (membero f vars)
       (== depth1 (cons 1 depth1-rest))
       (less-than-or-equalo depth1 max-depth)
       (bounded-untyped-expro depth1-rest vars arg))]
    
    [(fresh (test-expr then-expr else-expr depth1 depth1-rest)
       (== expr `(if ,test-expr then ,then-expr else ,else-expr))
       (== depth1 (cons 1 depth1-rest))
       (less-than-or-equalo depth1 max-depth)
       (bounded-untyped-expro depth1-rest vars test-expr)
       (bounded-untyped-expro depth1-rest vars then-expr)
       (bounded-untyped-expro depth1-rest vars else-expr))]))


In [43]:
(run* (expr) (bounded-untyped-expro (build-num 2) '(x) expr))

'((_.0 (num _.0)) x ((x _.0) (num _.0)) (x x) ((_.0 + _.1) (num _.0 _.1)) ((_.0 + x) (num _.0)) ((_.0 * _.1) (num _.0 _.1)) ((_.0 * x) (num _.0)) ((if _.0 then _.1 else _.2) (num _.0 _.1 _.2)) ((if _.0 then _.1 else x) (num _.0 _.1)) ((x + _.0) (num _.0)) ((x * _.0) (num _.0)) (x + x) (x * x) ((if _.0 then x else _.1) (num _.0 _.1)) ((if _.0 then x else x) (num _.0)) ((if x then _.0 else _.1) (num _.0 _.1)) ((if x then _.0 else x) (num _.0)) ((if x then x else _.0) (num _.0)) (if x then x else x))

In [45]:
(run 1 (vars) (bounded-untyped-expro (build-num 5) vars
'(if (p x) then (x * (f (x + 1))) else (f (f x)))))

'((p x f . _.0))

## 3.3 Type Checking, Type and Program Synthesis with Simple Types

**Exercise 3.3.1**

Implement a miniKanren relation `typed-expro`
such that
`(typed-expro context expr type)` is satisfied when `expr` is a valid expression of type `type` in context
`context`:

1. variable $x$ has type $T$ iff context $\Gamma$ explicitly has typing $x : T$
2. expressions $(e_1 + e_2)$ and $(e_1 \times e_2)$ have type $\mathsf{Int}$ and require both $e_1$ and $e_2$ to be of type $\mathsf{Int}$
3. function application $(f x)$ has type $T$ iff $f$ has function type $X \rightarrow T$ and $x$ has type $X$
4. expression if $e_1$ then $e_2$ else $e_3$ has type $T$ if $e_1$ has type $\mathsf{Bool}$ and both $e_2$ and $e_3$ have type $T$

Check your implementation:

Make sure that `typed-expro` works in **type checking** mode: it can check that given expression
is well-typed in a given context with given type:

```
(run* (q) (typed-expro sample-context '(if (f x) then (x + y) else y) 'Int))
; '(_.0)
(run* (q) (typed-expro sample-context '(if (f x) then (x + b) else y) 'Int))
; '()
```

Make sure that `typed-expro` works in **type inference** mode: it can check that given expression
is well-typed in a given context, but it can deduce the type automatically:

```
(run* (type) (typed-expro
`((1 . Int) . ,sample-context) ; we add constants explicitly to the context
'(if (f x) then (x + 1) else y) ; expression is given
type)) ; type is inferred
; '(Int)
```

Make sure that `typed-expro` works in **context inference** mode: it can infer which variables
(or constants) are used in a given expression and what their intended types are supposed to be
based on an expected type of the expression:

```
(run 1 (context) (typed-expro context '(if (f x) then (x + b) else y) 'Int))
; '(((f Int -> Bool) (x . Int) (b . Int) (y . Int) . _.0))
(run 1 (context) (typed-expro context '((f (g x)) + (g (f x))) 'Int))
; '(((f Int -> Int) (g Int -> Int) (x . Int) . _.0))
(run 1 (context type) (typed-expro context '(if (p x) then (f x) else x) type))
; '((((p _.0 -> Bool) (x . _.0) (f _.0 -> _.0) . _.1) _.0))
```

Make sure that `typed-expro` works in **program synthesis** mode: it can synthesise well-typed
expressions in a given context with a given type:

```
(run 6 (expr) (typed-expro sample-context expr 'Int))
; '(x y (x + x) (x * x) (x + y) (if b then x else x))
(run 6 (expr) (typed-expro sample-context expr 'Bool))
; '(b (f x) (f y) (f (x + x)) (if b then b else b) (f (x * x)))
(run 6 (expr) (typed-expro sample-context expr '(Int -> Int)))
; '((+ x) (+ y) (+ (x + x)) (+ (x * x)) (+ (x + y)) (+ (if b then x else x)))
```

In [311]:
(define (boolo t) (== t 'Bool))
(define (intoo t) (== t 'Int))
(define (arr-typeo t from-type to-type)
  (== t `(,from-type -> ,to-type)))

(define (lookupo x context type)
  (fresh (rest)
    (== context (cons (cons x type) rest))))

(define (typed-expro context expr type)
  (conde
    [(fresh (t)
       (lookupo expr context t)
       (== type t))]
    
    [(intoo type)
     (conde
       [(== expr 1)]
       [(== expr 0)]
       [(== expr -1)])]

    [(boolo type)
     (conde
       [(== expr 'b)]
       [(== expr #t)]
       [(== expr #f)])]
    
    [(fresh (e1 e2)
       (== expr `(,e1 + ,e2))
       (intoo type)
       (typed-expro context e1 'Int)
       (typed-expro context e2 'Int))]
    
    [(fresh (e1 e2)
       (== expr `(,e1 * ,e2))
       (intoo type)
       (typed-expro context e1 'Int)
       (typed-expro context e2 'Int))]
    
    [(fresh (f x from-type)
       (== expr `(,f ,x))
       (fresh (func-type)
         (arr-typeo func-type from-type type)
         (typed-expro context f func-type)
         (typed-expro context x from-type)))]
    
    [(fresh (e1 e2 e3)
       (== expr `(if ,e1 then ,e2 else ,e3))
       (typed-expro context e1 'Bool)
       (typed-expro context e2 type)
       (typed-expro context e3 type))]
    ))

(define sample-context 
  `((x . Int) (y . Int) (f . (Int -> Bool)) (g . (Int -> Int))))

In [51]:
(run* (q) (typed-expro sample-context '(if (f x) then (x + y) else y) 'Int))

'()

In [53]:
(run* (q) (typed-expro sample-context '(if (f x) then (x + b) else y) 'Int))

'()

In [55]:
(run* (type) (typed-expro
`((1 . Int) . ,sample-context)
'(if (f x) then (x + 1) else y)
type))

'()

In [57]:
(run 1 (context) (typed-expro context '(if (f x) then (x + b) else y) 'Int))
; '(((f Int -> Bool) (x . Int) (b . Int) (y . Int) . _.0))

'((((if (f x) then (x + b) else y) . Int) . _.0))

In [59]:
(run 1 (context) (typed-expro context '((f (g x)) + (g (f x))) 'Int))
; '(((f Int -> Int) (g Int -> Int) (x . Int) . _.0))

'(((((f (g x)) + (g (f x))) . Int) . _.0))

In [61]:
(run 1 (context type) (typed-expro context '(if (p x) then (f x) else x) type))
; '((((p _.0 -> Bool) (x . _.0) (f _.0 -> _.0) . _.1) _.0))

'(((((if (p x) then (f x) else x) . _.0) . _.1) _.0))

In [63]:
(run 6 (expr) (typed-expro sample-context expr 'Int))
; '(x y (x + x) (x * x) (x + y) (if b then x else x))

'(x 1 0 -1 (x + x) (x + 1))

In [65]:
(run 6 (expr) (typed-expro sample-context expr 'Bool))
; '(b (f x) (f y) (f (x + x)) (if b then b else b) (f (x * x)))

'(b #t #f (if b then b else b) (if b then b else #t) (if b then b else #f))

In [ ]:
(run 6 (expr) (typed-expro sample-context expr '(Int -> Int)))
; '((+ x) (+ y) (+ (x + x)) (+ (x * x)) (+ (x + y)) (+ (if b then x else x)))

## 3.4 Interlude: Lists as Sets

The next section may require the use of lists as sets, so in this section we implement more interesting
helpers. Many of these are straightforward to implement in one direction, but you should make the
effort to allow them to work properly in different modes. To achieve that, it may be useful to make
sure that you incrementally learn more about all arguments of a relation. This way, you sometimes
can ensure that regardless of the mode the logical engine will make progress towards the solution.

**Exercise 3.4.1**

Implement a miniKanren relation `remove-firsto`
such that
`(remove-firsto x before after)` is satisfied when `after` is a list containing at most one less occurrence of `x` (first one is removed) compared to `before` (but otherwise the same). Check your
implementation (at least two modes):

```
(run* (ys) (remove-firsto 'o '(h e l l o w o r l d) ys))
; '((h e l l w o r l d))
(run* (xs) (remove-firsto 0 xs '(1 2 3)))
; '((0 1 2 3) (1 0 2 3) (1 2 0 3) (1 2 3) (1 2 3 0))
```

In [67]:
(define (remove-firsto x before after)
  (conde
   [(fresh (rest)
      (== before (cons x rest))
      (== after rest))]
   
   [(fresh (first rest before-rest)
      (== before (cons first rest))
      (== after (cons first before-rest))
      (remove-firsto x rest before-rest))]
   
   [(== before after)
    (absent x before)]))

(define (absent x lst)
  (conde
   [(== lst '())]
   [(fresh (first rest)
      (== lst (cons first rest))
      (=/= first x)
      (absent x rest))]
   [(== lst #f)]))

In [69]:
(run* (ys) (remove-firsto 'o '(h e l l o w o r l d) ys))

'((h e l l w o r l d) (h e l l o w r l d) (h e l l o w o r l d) (h e l l o w o r l d) (h e l l o w o r l d) (h e l l o w o r l d))

In [71]:
(run* (xs) (remove-firsto 0 xs '(1 2 3)))

'((0 1 2 3) (1 0 2 3) (1 2 3) (1 2 0 3) (1 2 3) (1 2 3) (1 2 3 0) (1 2 3))

**Exercise 3.4.2** 

Implement a miniKanren relation `remove-allo`
such that
`(remove-allo x before after)` is satisfied when `after` is the same as `before` with all occurrences
of `x` removed. Check your implementation (one mode is enough):

```
(run* (ys) (removeo 'a '(a b r a c a d a b r a) ys))
; '((b r c d b r))
```

In [165]:
(define remove-allo
  (lambda (x before after)
    (conde
     [(== '() before) (== '() after)]
     
     [(fresh (tail)
        (== `(,x . ,tail) before)
        (remove-allo x tail after))]
     
     [(fresh (head tail rest)
        (== `(,head . ,tail) before)
        (=/= head x)
        (== `(,head . ,rest) after)
        (remove-allo x tail rest))])))


In [167]:
(run* (ys) (remove-allo 'a '(a b r a c a d a b r a) ys))

'((b r c d b r))

**Exercise 3.4.3**

Implement a miniKanren relation `remove-duplicateso`
such that
`(remove-duplicateso before after)` is satisfied when `after` is the same as `before` with all duplicates removed. Check your implementation (one mode is enough):

```
(run* (ys) (remove-duplicateso '(a b r a c a d a b r a) ys))
; '((a b r c d))
```

In [391]:
(define (remove-duplicateso before after)
  (conde
    [(== '() before) (== '() after)]
    
    [(fresh (first rest unique-rest)
       (== (cons first rest) before)
       
       (remove-duplicateso rest unique-rest)
       
       (not-membero first unique-rest)
       
       (== (cons first unique-rest) after))]
    
    [(fresh (first rest unique-rest)
       (== (cons first rest) before)
       (remove-duplicateso rest unique-rest)
       (membero first unique-rest)
       (== unique-rest after))]))

(define (not-membero x ls)
  (conde
    [(== '() ls)]
    [(fresh (first rest)
       (== (cons first rest) ls)
       (=/= x first)
       (not-membero x rest))]))

(run* (ys) (remove-duplicateso '(a b r a c a d a b r a) ys))

'((c d b r a))

**Exercise 3.4.4**

Implement a miniKanren relation `uniono`
such that
`(uniono xs ys xys)` is satisfied when `xys` is the union (without duplicates) of `xs` and `ys`. Check your
implementation (one mode is enough):

```
(run* (ys) (uniono '(h e l l o) '(w o r l d) ys))
; '((h e l o w r d))
```

In [192]:
(define (membero x l)
  (fresh (h t)
    (== (cons h t) l)
    (conde
      [(== h x)]
      [(membero x t)])))

(define (not-membero x l)
  (conde
    [(== '() l)]
    [(fresh (h t)
       (== (cons h t) l)
       (=/= h x)
       (not-membero x t))]))

(define (uniono xs ys xys)
  (conde
    [(== '() xs) (== ys xys)]
    [(fresh (a d res)
       (== (cons a d) xs)
       (conde
         [(membero a ys) (uniono d ys xys)]
         [(fresh ()
            (not-membero a ys)
            (uniono d ys res)
            (== xys (cons a res)))]))]))

(run* (ys) (uniono '(h e l l o) '(w o r l d) ys))


'((h e w o r l d))

**Exercise 3.4.5**

Implement a miniKanren relation `same-lengtho`
such that
`(same-lengtho xs ys)` is satisfied when `xs` and `ys` are both lists of the same length. Check your
implementation (all modes should work productively):

```
(run* (xs) (same-lengtho xs '(1 2 3)))
; '((_.0 _.1 _.2))
(run* (ys) (same-lengtho '(1 2 3) ys))
; '((_.0 _.1 _.2))
(run 3 (xs ys) (same-lengtho xs ys))
; '((() ()) ((_.0) (_.1)) ((_.0 _.1) (_.2 _.3)))
```

In [203]:
(define (same-lengtho xs ys)
  (conde
    [(== '() xs) (== '() ys)]
    [(fresh (xa xd ya yd)
       (== (cons xa xd) xs)
       (== (cons ya yd) ys)
       (same-lengtho xd yd))]))

In [205]:
(run* (xs) (same-lengtho xs '(1 2 3)))


'((_.0 _.1 _.2))

In [207]:
(run* (ys) (same-lengtho '(1 2 3) ys))

'((_.0 _.1 _.2))

In [209]:
(run 3 (xs ys) (same-lengtho xs ys))

'((() ()) ((_.0) (_.1)) ((_.0 _.1) (_.2 _.3)))

**Exercise 3.4.6**

Implement a miniKanren relation `equiv-seto`
such that
`(equiv-seto xs ys)` is satisfied when `xs` and `ys` are both lists that represent sets with the same
elements. You may assume that `xs` and `ys` do not contain duplicates. Check your implementation (all
modes should work productively, the order of results may differ):

```
(run* (xs) (equiv-seto xs '(1 2 3)))
; '((1 2 3) (1 3 2) (2 1 3) (2 3 1) (3 1 2) (3 2 1))
(run* (xs) (equiv-seto '(1 2 3) xs))
; '((1 2 3) (1 3 2) (2 1 3) (3 1 2) (2 3 1) (3 2 1))
```

In [231]:
(define (membero x l)
  (fresh (h t)
    (== (cons h t) l)
    (conde
      [(== h x)]
      [(membero x t)])))

(define (removeo x l out)
  (conde
    [(== '() l) (== '() out)]
    [(fresh (h t res)
       (== (cons h t) l)
       (conde
         [(== h x) (== t out)]
         [(=/= h x)
          (removeo x t res)
          (== (cons h res) out)]))]))

(define (equiv-seto xs ys)
  (conde
    [(== '() xs) (== '() ys)]
    [(fresh (a d ys-rest)
       (== (cons a d) xs)
       (membero a ys)
       (removeo a ys ys-rest)
       (equiv-seto d ys-rest))]))


In [233]:
(run* (xs) (equiv-seto xs '(1 2 3)))

'((1 2 3) (1 3 2) (2 1 3) (3 1 2) (2 3 1) (3 2 1))

In [242]:
(run 6 (xs) (equiv-seto '(1 2 3) xs))

'((1 2 3) (1 3 2) (2 1 3) (3 1 2) (2 3 1) (3 2 1))

## 3.5 More Program Synthesis with Simple Types

As you have seen previously, we can already synthesize programs, but they tend to be rather boring.
For examples, despite the “fair” search in miniKanren, we still get a lot of expressions that use the
same variable (usually the one that appears earlier in the context). To improve the state of things a
little, we will demand that all variables in the context are actually used in the generated expression.
Instead of rewriting all our previous code, we will postulate this requirement post-factum.

**Exercise 3.5.1**

Implement a miniKanren relation `free-varso`
such that
`(free-varso expr vars)` is satisfied when `expr` is a valid expression with exactly the variables from
`vars`. This is similar to `untyped-expro`
, except now the list of variables must be exhaustive and not
contain any duplicates. Check your implementation:

```
(run* (vars) (free-varso '(if (f x) then (x + y) else x) vars))
; '((f x y))
```

In [321]:

(define (symbolo x)
  (conde
    [(=/= x 'if) (=/= x 'then) (=/= x 'else) (=/= x '+)]))

(define (free-varso expr vars)
  (conde
    [(symbolo expr) (== vars (list expr))]
    
    [(fresh (test conseq alt test-vars conseq-vars alt-vars temp-vars)
       (== `(if ,test then ,conseq else ,alt) expr)
       (free-varso test test-vars)
       (free-varso conseq conseq-vars)
       (free-varso alt alt-vars)
       (uniono test-vars conseq-vars temp-vars)
       (uniono temp-vars alt-vars vars))]
    
    [(fresh (x y x-vars y-vars)
       (== `(,x + ,y) expr)
       (free-varso x x-vars)
       (free-varso y y-vars)
       (uniono x-vars y-vars vars))]
    
    [(== '() expr) (== vars '())]))

(define (membero x l)
  (fresh (h t)
    (== (cons h t) l)
    (conde
      [(== h x)]
      [(membero x t)])))

(define (uniono xs ys xys)
  (conde
    [(== '() xs) (== ys xys)]
    [(fresh (a d res)
       (== (cons a d) xs)
       (conde
         [(membero a ys) (uniono d ys xys)]
         [(not-membero a ys)
          (uniono d ys res)
          (== xys (cons a res))]))]))

(define (not-membero x l)
  (conde
    [(== '() l)]
    [(fresh (h t)
       (== (cons h t) l)
       (=/= h x)
       (not-membero x t))]))

(run* (vars) (free-varso '(if (f x) then (x + y) else x) vars))


'(((if (f x) then (x + y) else x)) ((f x) (x + y) x) ((f x) y x))

Next step is easy — ensure that context consists precisely of a (permutation of) the list of free
variables of the generated expression. This way, we will only generate programs that use *all* context
information.

**Exercise 3.5.2**

Implement a miniKanren relation `no-unused-typed-expro`
such that
`(no-unused-typed-expro context expr type)` is satisfied when `expr` is a valid expression of type
`type` that uses **all** variables from the context context at least once.

*Hint: use a combination of previously defined relations.*

Check your implementation:

Make sure that `no-unused-typed-expro` works in **program synthesis** mode for relatively simple programs:

```
(define local-compose-context
  '((f . (b -> c))
    (g . (a -> b))
    (x . a)))
(run 1 (expr) (no-unused-typed-expro local-compose-context expr 'c))
; '((f (g x)))

(define local-compose-context-2
  '((f . (a -> a))
    (g . (a -> a))
    (x . a)))
(run 5 (expr) (no-unused-typed-expro local-compose-context-2 expr 'a))
; '((f (g x)) (g (f x)) (f (f (g x))) (f (g (f x))) (g (f (f x))))
```

In [373]:
(define (appendo l s out)
  (conde
    [(== l '()) (== s out)]
    [(fresh (a d res)
       (== l (cons a d))
       (== out (cons a res))
       (appendo d s res))])) 

(define (membero x lst)
  (fresh (head tail)
    (conde
      [(== lst '()) fail]
      [(== lst (cons x tail))]
      [(fresh (rest)
         (== lst (cons head rest))
         (=/= head x)
         (membero x rest))])))

(define (lookupo x ctx type)
  (fresh (rest-ctx name rest-type)
    (conde
      [(== ctx '()) fail]
      [(fresh ()
         (== ctx (cons (cons name type) rest-ctx))
         (conde
           [(== name x) (== type rest-type)]
           [(=/= name x) (lookupo x rest-ctx type)]))])))

(define (collect-varso expr vars)
  (conde
    [(symbolo expr) (== vars (list expr))]
    
    [(fresh (f arg f-vars arg-vars)
       (== expr `(,f ,arg))
       (collect-varso f f-vars)
       (collect-varso arg arg-vars)
       (appendo f-vars arg-vars vars))]
    
    [(== expr '()) (== vars '())]))

(define (context-vars-usedo ctx vars)
  (conde
    [(== ctx '())]
    [(fresh (name type rest-ctx)
       (== ctx (cons (cons name type) rest-ctx))
       (membero name vars)
       (context-vars-usedo rest-ctx vars))])) 

(define (no-unused-typed-expro ctx expr result-type)
  (fresh (used-vars)
    (collect-varso expr used-vars)
    (context-vars-usedo ctx used-vars)
    (conde
      [(symbolo expr)
       (lookupo expr ctx result-type)]
      
      [(fresh (f arg f-type arg-type)
         (== expr `(,f ,arg))
         (lookupo f ctx `(,arg-type -> ,result-type))
         (no-unused-typed-expro ctx f `(,arg-type -> ,result-type))
         (no-unused-typed-expro ctx arg arg-type))])))

(define local-compose-context
  '((f . (b -> c))
    (g . (a -> b))
    (x . a)))

(define local-compose-context-2
  '((f . (a -> a))
    (g . (a -> a))
    (x . a)))

Make sure that `no-unused-typed-expro`
can synthesize the implementation of `map` given a small
context and a small hint:

```
(define local-map-context
  '((empty? . ([a] -> Bool))
    (empty . [b])
    (cons . (b -> ([b] -> [b])))
    (first . ([a] -> a))
    (rest . ([a] -> [a]))
    (map . ((a -> b) -> ([a] -> [b])))
    (f . (a -> b))
    (xs . [a])))
(run 1 (expr)
    (fresh (e1 e2)
        (== expr `(if (empty? xs) then ,e1 else ,e2)))
    (no-unused-typed-expro local-map-context expr '[b]))
; '((if (empty? xs) then empty else ((cons (f (first xs))) ((map f) (rest xs)))))
```

In [379]:
(define (membero x ls)
  (fresh (a d)
    (== ls (cons a d))
    (conde
      [(== a x)]
      [(=/= a x) (membero x d)])))

(define (lookupo x ctx type)
  (fresh (rest-ctx name rest-type)
    (== ctx (cons (cons name type) rest-ctx))
    (conde
      [(== name x) (== type rest-type)]
      [(=/= name x) (lookupo x rest-ctx type)])))

(define (collect-varso expr vars)
  (conde
    [(symbolo expr) (== vars (list expr))]
    
    [(fresh (f arg f-vars arg-vars)
       (== expr `(,f ,arg))
       (collect-varso f f-vars)
       (collect-varso arg arg-vars)
       (appendo f-vars arg-vars vars))]
    
    [(fresh (test then-branch else-branch test-vars then-vars else-vars)
       (== expr `(if ,test then ,then-branch else ,else-branch))
       (collect-varso test test-vars)
       (collect-varso then-branch then-vars)
       (collect-varso else-branch else-vars)
       (appendo test-vars then-vars vars1)
       (appendo vars1 else-vars vars))]
    
    [(== expr '()) (== vars '())]))

(define (context-vars-usedo ctx vars)
  (conde
    [(== ctx '())]
    [(fresh (name type rest-ctx)
       (== ctx (cons (cons name type) rest-ctx))
       (membero name vars)
       (context-vars-usedo rest-ctx vars))])) 

(define (no-unused-typed-expro ctx expr result-type)
  (fresh (used-vars)
    (collect-varso expr used-vars)
    (context-vars-usedo ctx used-vars)
    (conde
      [(symbolo expr)
       (lookupo expr ctx result-type)]
      
      [(fresh (f arg f-type arg-type)
         (== expr `(,f ,arg))
         (lookupo f ctx `(,arg-type -> ,result-type))
         (no-unused-typed-expro ctx f `(,arg-type -> ,result-type))
         (no-unused-typed-expro ctx arg arg-type))]
      
      [(fresh (test then-branch else-branch test-type then-type)
         (== expr `(if ,test then ,then-branch else ,else-branch))
         (lookupo test ctx 'Bool)
         (no-unused-typed-expro ctx test 'Bool)
         (no-unused-typed-expro ctx then-branch result-type)
         (no-unused-typed-expro ctx else-branch result-type))])))

(define local-map-context
  '((empty? . ([a] -> Bool))
    (empty . [b])
    (cons . (b -> ([b] -> [b])))
    (first . ([a] -> a))
    (rest . ([a] -> [a]))
    (map . ((a -> b) -> ([a] -> [b])))
    (f . (a -> b))
    (xs . [a])))


Use `no-unused-typed-expro`
to synthesize the implementation of `append` from the following
context and a small hint (in reasonable time):

```
(define local-append-context
  '((xs . [a])
    (ys . [a])
    (empty? . ([a] -> Bool))
    (cons . (a -> ([a] -> [a])))
    (first . ([a] -> a))
    (rest . ([a] -> [a]))
    (append . ([a] -> ([a] -> [a])))))
(run 10 (expr)
    (fresh (e2)
        (== expr `(if (empty? xs) then ys else ,e2)))
(no-unused-typed-expro local-append-context expr '[a]))
; '(...
; (if (empty? xs) then ys else (cons (first xs) (append (rest xs) ys)))
; ...)
```

In [383]:
(define (membero x ls)
  (fresh (a d)
    (== ls (cons a d))
    (conde
      [(== a x)]
      [(=/= a x) (membero x d)])))

(define (lookupo x ctx type)
  (fresh (rest-ctx name rest-type)
    (== ctx (cons (cons name type) rest-ctx))
    (conde
      [(== name x) (== type rest-type)]
      [(=/= name x) (lookupo x rest-ctx type)])))

(define (collect-varso expr vars)
  (conde
    [(symbolo expr) (== vars (list expr))]
    
    [(fresh (f arg f-vars arg-vars)
       (== expr `(,f ,arg))
       (collect-varso f f-vars)
       (collect-varso arg arg-vars)
       (appendo f-vars arg-vars vars))]
    
    [(fresh (test then-branch else-branch test-vars then-vars else-vars)
       (== expr `(if ,test then ,then-branch else ,else-branch))
       (collect-varso test test-vars)
       (collect-varso then-branch then-vars)
       (collect-varso else-branch else-vars)
       (appendo test-vars then-vars vars1)
       (appendo vars1 else-vars vars))]
    
    [(== expr '()) (== vars '())]))

(define (context-vars-usedo ctx vars)
  (conde
    [(== ctx '())]
    [(fresh (name type rest-ctx)
       (== ctx (cons (cons name type) rest-ctx))
       (membero name vars)
       (context-vars-usedo rest-ctx vars))])) 

(define (no-unused-typed-expro ctx expr result-type)
  (fresh (used-vars)
    (collect-varso expr used-vars)
    (context-vars-usedo ctx used-vars)
    (conde
      [(symbolo expr)
       (lookupo expr ctx result-type)]
      
      [(fresh (f arg f-type arg-type)
         (== expr `(,f ,arg))
         (lookupo f ctx `(,arg-type -> ,result-type))
         (no-unused-typed-expro ctx f `(,arg-type -> ,result-type))
         (no-unused-typed-expro ctx arg arg-type))]
      
      [(fresh (test then-branch else-branch test-type then-type)
         (== expr `(if ,test then ,then-branch else ,else-branch))
         (lookupo test ctx 'Bool)
         (no-unused-typed-expro ctx test 'Bool)
         (no-unused-typed-expro ctx then-branch result-type)
         (no-unused-typed-expro ctx else-branch result-type))])))

(define local-append-context
  '((xs . [a])
    (ys . [a])
    (empty? . ([a] -> Bool))
    (cons . (a -> ([a] -> [a])))
    (first . ([a] -> a))
    (rest . ([a] -> [a]))
    (append . ([a] -> ([a] -> [a])))))